In [1]:
import pandas as pd
import glob
import json

In [2]:
json_folder = glob.glob("data/raw_json/*/*.json")

In [3]:
sesiones = []
votaciones = []
grupos_parlamentarios = {}
diputados = {}
votos = []
grupo_id_counter = 0
diputado_id_counter = 0

In [4]:
for json_file in json_folder:
    with open(json_file, "r", encoding="utf-8") as json_file:
        # Load the JSON file
        # Assuming the JSON file contains a list of dictionaries
        # If the structure is different, adjust accordingly
        # For example, if it's a single dictionary, you can just load it directly
        #
        data = json.load(json_file)
        # Extract session data
        sesion_id = data["informacion"]["sesion"]
        if sesion_id not in [s["id_sesion"] for s in sesiones]:
            sesiones.append({
                "id_sesion": sesion_id,
                "fecha": data["informacion"]["fecha"],
                "numero_sesion": data["informacion"]["sesion"]
            })

        # Extract votacion data
        votacion_id = len(votaciones) + 1
        votaciones.append({
            "id_votacion": votacion_id,
            "id_sesion": sesion_id,
            "numero_votacion": data["informacion"]["numeroVotacion"],
            "titulo": data["informacion"]["titulo"],
            "texto_expediente": data["informacion"]["textoExpediente"],
            "presentes": data["totales"]["presentes"],
            "a_favor": data["totales"]["afavor"],
            "en_contra": data["totales"]["enContra"],
            "abstenciones": data["totales"]["abstenciones"],
            "no_votan": data["totales"]["noVotan"]
        })

        # Extract grupos parlamentarios and diputados
        for votacion in data["votaciones"]:
            grupo = votacion.get("grupo", "")
            if grupo == "":
                print(f"Grupo vacío en la votación: {votacion}")
                continue  # Skip empty groups
            if grupo not in grupos_parlamentarios:
                grupos_parlamentarios[grupo] = grupo_id_counter
                grupo_id_counter += 1

            diputado = votacion.get("diputado", "Unknown")
            if diputado not in diputados:
                diputados[diputado] = {
                    "id_diputado": diputado_id_counter,
                    "nombre": diputado,
                    "id_grupo": grupos_parlamentarios[grupo]
                }
                diputado_id_counter += 1

            # Extract votos
            votos.append({
                "id_voto": len(votos) + 1,
                "id_votacion": votacion_id,
                "id_diputado": diputados[diputado]["id_diputado"],
                "asiento": votacion.get("asiento", ""),
                "voto": votacion["voto"]
            })

# Convert data to DataFrames
sesiones_df = pd.DataFrame(sesiones)
votaciones_df = pd.DataFrame(votaciones)
grupos_parlamentarios_df = pd.DataFrame([
{"id_grupo": id_grupo, "nombre": nombre}
for nombre, id_grupo in grupos_parlamentarios.items()
])
diputados_df = pd.DataFrame(diputados.values())
votos_df = pd.DataFrame(votos)

Grupo vacío en la votación: {'asiento': '1804', 'diputado': 'Conesa Coma, Ignasi', 'grupo': '', 'voto': 'Sí'}
Grupo vacío en la votación: {'asiento': '1804', 'diputado': 'Conesa Coma, Ignasi', 'grupo': '', 'voto': 'Sí'}
Grupo vacío en la votación: {'asiento': '2511', 'diputado': 'Santana Perera, Noemí', 'grupo': '', 'voto': 'Sí'}
Grupo vacío en la votación: {'asiento': '2511', 'diputado': 'Santana Perera, Noemí', 'grupo': '', 'voto': 'No'}


In [5]:
# format date of sesiones_df to YYYY-MM-DD
sesiones_df["fecha"] = pd.to_datetime(sesiones_df["fecha"], format="%d/%m/%Y").dt.strftime("%Y-%m-%d")

In [6]:
PP = 'PP'
PSOE = 'PSOE'
VOX = 'VOX'
ERC = 'ERC'
JXCAT = 'JxCAT'
MIXTO = 'Mixto'
SUMAR = 'Sumar'
BILDU = 'EH Bildu'
PNV = 'EAJ-PNV'
ERC = 'ERC'

In [7]:
grupos_parlamentarios_df['nombre'] = grupos_parlamentarios_df['nombre'].replace(
    {
        'GP': PP,
        'GVOX': VOX,
        'GMx': MIXTO,
        'GJxCAT': JXCAT,
        'GS': PSOE,
        'GSUMAR': SUMAR,
        'GEH Bildu': BILDU,
        'GV (EAJ-PNV)': PNV,
        'GR': ERC,
    }
)
# crea una nueva columna con el color de cada grupo parlamentario
grupos_parlamentarios_df['color'] = grupos_parlamentarios_df['nombre'].replace(
    {
        PP: '#1E4B8F',
        PSOE: '#E30713',
        VOX: '#63BE21',
        ERC: '#FFB232',
        JXCAT: '#20C0B2',
        MIXTO: '#A0A0A0',
        SUMAR: '#E51C55',
        BILDU: '#008000',
        PNV: '#007A33'
    }
)
grupos_parlamentarios_df['order'] = grupos_parlamentarios_df['nombre'].replace(
    {
        PP: 7,
        PSOE: 4,
        VOX: 8,
        ERC: 2,
        JXCAT: 6,
        MIXTO: 0,
        SUMAR: 3,
        BILDU: 1,
        PNV: 5
    }
)

## Detección proponentes

In [8]:
import re
# Tries to detect which groupo parlamentario proposed the votacion from column texto_expediente
def generate_proponentes(votaciones_df):

    regex_grupos = [
        # Popular
        (r'popular', PP),
        (r'sumar', SUMAR),
        (r'vox', VOX),
        (r'republicano', ERC),
        (r'euskal herria bildu', BILDU),
        (r'mixto', MIXTO),
        (r'eaj-pnv', PNV),
        (r'junts per catalunya', JXCAT),
        (r'socialista', PSOE),
    ]
    proponents = []
    for index, row in votaciones_df.iterrows():
        # Extract the text from first mention to "Grupo(s) parlamentario(s) until"
        texto_expediente = row['texto_expediente']

        for regex, grupo in regex_grupos:
            if re.search(regex, texto_expediente, re.IGNORECASE):
                proponents.append({
                    'id_votacion': row['id_votacion'],
                    'proponente': grupo
                })
    return pd.DataFrame(proponents)

proponentes_df = generate_proponentes(votaciones_df)

In [9]:
proponentes_df = proponentes_df.merge(grupos_parlamentarios_df, how='left', left_on='proponente', right_on='nombre', suffixes=('', '_grupo'))[['id_votacion', 'id_grupo']]

## Detección subcategorias

In [ ]:
df_subcategorias = pd.read_csv("subcategorias.csv", encoding="utf-8")

In [ ]:
# Diccionario básico de palabras clave por subcategoría
with open("keywords_por_subcategoria.json", "r", encoding="utf-8") as f:
    keywords_por_subcategoria = json.load(f)

def asignar_subcategorias(texto_expediente):
    texto = texto_expediente.lower()
    ids_detectados = set()
    
    for subcat, keywords in keywords_por_subcategoria.items():
        if any(kw in texto for kw in keywords):
            id_sub = df_subcategorias[df_subcategorias["nombre_subcategoria"] == subcat]["id_subcategoria"].values
            if len(id_sub):
                ids_detectados.add(int(id_sub[0]))
    
    return list(ids_detectados) if ids_detectados else None

In [13]:
votaciones_df['subcategorias'] = votaciones_df['texto_expediente'].apply(asignar_subcategorias)

In [14]:
votaciones_df['subcategorias'].notna().sum() / votaciones_df.shape[0] * 100

37.236962488563584

In [15]:
votaciones_subcategorias = votaciones_df.explode('subcategorias')[['id_votacion', 'subcategorias']].query("subcategorias.notna()").rename(columns={'subcategorias': 'id_subcategoria'}).drop_duplicates().reset_index(drop=True)

In [16]:
votaciones_df.drop(columns=['subcategorias'], inplace=True)

In [17]:
# Save DataFrames to CSV files in data/processed folder (create if not exists)
import os
os.makedirs("data/processed", exist_ok=True)
sesiones_df.to_csv("data/processed/sesiones.csv", index=False, encoding="utf-8")
votaciones_df.to_csv("data/processed/votaciones.csv", index=False, encoding="utf-8")
grupos_parlamentarios_df.to_csv("data/processed/grupos_parlamentarios.csv", index=False, encoding="utf-8")
diputados_df.to_csv("data/processed/diputados.csv", index=False, encoding="utf-8")
votos_df.to_csv("data/processed/votos.csv", index=False, encoding="utf-8")
proponentes_df.to_csv("data/processed/proponentes.csv", index=False, encoding="utf-8")
votaciones_subcategorias.to_csv("data/processed/votaciones_subcategorias.csv", index=False, encoding="utf-8")